# Multi-channel PolyConvNet

In [ ]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt

from keras import Model
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Conv1D, Flatten, Concatenate, Input

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Load dataset
data = 'C' # ****** specify dataset (A / B / C / D) ******
dataset = pd.read_csv(f'./data/data_{data}.csv')
dataset_name='dataSet'+data

In [ ]:
# Load kernels
kernel_matrix=[]
kernels = open(f'./features/{dataset_name}.kern').readlines()
for i in range(len(kernels)):
    kernel = [float(i) for i in kernels[i].strip().split()]
    kernel_matrix.append(kernel)
    
# Load Principle Component 1
PC = pd.DataFrame(pd.read_csv(f'./features/dataset{data}_principal_component_analysis.csv').iloc[:,1:4])

# Load sliding window features
window_size = 2 # ****** specify window size ******

n_channel = 2**window_size

X = np.loadtxt(f"./features/sliding_window_{window_size}_dataset_{data}.txt")
X = X.reshape(X.shape[0], X.shape[1] // n_channel , n_channel)
interaction_param, y = dataset['interaction_parameter'], dataset['lamellar_period']

In [ ]:
# train / test split
train_idx = np.random.choice(range(len(X)), size=int(0.8*(len(X))), replace=False)
X_train = X[train_idx]
y_train = np.array([y[i] for i in train_idx])

test_idx = list(set(range(len(X)))-set(train_idx))
X_test = X[test_idx]
y_test = np.array([y[i] for i in test_idx])

# auxInput
#interaction_kernel_train = np.array([[interaction_param[i]]+kernel_matrix[i] for i in train_idx])
#interaction_kernel_test = np.array([[interaction_param[i]]+kernel_matrix[i] for i in test_idx])
# Add Principle Components
interaction_kernel_train = np.array([[interaction_param[i]]+kernel_matrix[i]+list(PC.iloc[i,:]) for i in train_idx])
interaction_kernel_test = np.array([[interaction_param[i]]+kernel_matrix[i]+list(PC.iloc[i,:]) for i in test_idx])

In [ ]:
input_b = Input(shape=(interaction_kernel_train.shape[1]))

input = Input(shape =(32-window_size,n_channel)) 

x = Conv1D(32-window_size, 2, activation="relu", input_shape=(32-window_size,n_channel))(input)
x = Flatten()(x)

x = Concatenate()([x,input_b])
x = Flatten()(x)

x = Dense(units = 300, activation ='relu')(x)
x = Dense(units = 110, activation ='relu')(x)

regression = Dense(units = 1, activation='relu', name='regression')(x)

model = Model(inputs=[input,input_b], outputs=regression)

model.compile(loss='mse', optimizer=Adam(lr=0.001))
model.summary()

In [ ]:
# model training
epochs = 500
history = model.fit([X_train, interaction_kernel_train], y_train, batch_size=12,epochs=epochs, verbose=0)

In [ ]:
# making prediction
ypred_test = model.predict([X_test, interaction_kernel_test])
model_mse = mean_squared_error(y_test, ypred_test)
print("MSE: %.4f" % model_mse)

In [ ]:
# plot result
fig = plt.figure()
fig.patch.set_facecolor('white')
fig.patch.set_alpha(1)
ax = fig.add_subplot(111)

plt.title(f'MSE = {model_mse:.4f}.  Epochs = {epochs}.  R^2 = {r2_score(ypred_test.flatten(), y_test):.4f}') # MSE
plt.scatter(ypred_test, y_test, s=5, color="blue") # x_axis, y_axis, ...
plt.xlabel("Predicted")
plt.ylabel("Actual")

lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

plt.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
plt.xlim(lims)
plt.ylim(lims)
plt.savefig(f'./predictions/pred_{data}')

# save model
model.save(f"./models/multi_channel_polycovnet_{data}")